In [ ]:
from pathlib import Path
from typing import Iterable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

from common.data import load_data

sns.set_theme()

In [ ]:
EOL_CAPACITY = 1.4

In [ ]:
base_path = Path("../data/5. Battery Data Set/1. BatteryAgingARC-FY08Q4")

def make_paths(names: Iterable[str]):
    return [
        base_path.joinpath(name)
        for name in names
    ]

train_paths = make_paths(["B0005.mat", "B0006.mat"])
valid_paths = make_paths(["B0007.mat"])
test_paths = make_paths(["B0018.mat"])

In [ ]:
def load_paths(paths: Iterable[Path]) -> pd.DataFrame:
    data = pd.concat({Path(p).stem: load_data(p, "discharge") for p in paths})
    data.index.names = ["file", "index"]
    return data

In [ ]:
train_data = load_paths(train_paths)
valid_data = load_paths(valid_paths)
test_data = load_paths(test_paths)

In [ ]:
def window_dataframe(df: pd.DataFrame, size: int) -> pd.DataFrame:
    windows = []
    for s in range(size):
        shifted = df.shift(s)
        shifted.columns = shifted.columns.map(lambda c: f"{c}_b{s}")
        windows.append(shifted)
    return pd.concat(windows, axis=1)

def process_label(series: pd.Series, rolling_size: int) -> pd.Series:
    res = series.rolling(rolling_size).mean()
    res = res.shift(-1)
    return res

def process_data(data: pd.DataFrame, window_size: int, rolling_size=1):
    X = []
    y = []
    etc = []

    for _, group_df in data.groupby("file"):
        operation_df = group_df.groupby("operation_id").agg({
            "Time": ["max"],
            "Capacity": ["first"],
        })
        operation_df.columns = operation_df.columns.map(lambda c: "_".join(c))

        capacity_change = process_label(operation_df["Capacity_first"], rolling_size)
        time_change = process_label(operation_df["Time_max"], rolling_size)

        is_alive = operation_df["Capacity_first"] > EOL_CAPACITY
        alive_cycles = is_alive.sum()
        rul_cycles = -(np.arange(len(is_alive)) - alive_cycles)

        data_slice = slice(max(rolling_size, window_size) -1, -1)
        X_win = window_dataframe(operation_df, window_size).iloc[data_slice]
        
        X.append(X_win)
        y.append(pd.DataFrame({
            "time": time_change.iloc[data_slice],
            "cap": capacity_change.iloc[data_slice],
            "rul": rul_cycles[data_slice],
        }))
        etc.append(pd.DataFrame({
            "rul": rul_cycles[data_slice],
            "real_cap": operation_df["Capacity_first"].iloc[data_slice],
        }))

    X = pd.concat(X, ignore_index=True)
    y = pd.concat(y, ignore_index=True)
    etc = pd.concat(etc, ignore_index=True)

    return X, y, etc

In [ ]:
window_size = 5
train_X, train_y, train_etc = process_data(train_data, window_size=window_size)
valid_X, valid_y, valid_etc = process_data(valid_data, window_size=window_size)
test_X, test_y, test_etc = process_data(test_data, window_size=window_size)

In [ ]:
norm = StandardScaler().fit(train_X)
train_X_norm = norm.transform(train_X)
valid_X_norm = norm.transform(valid_X)
test_X_norm = norm.transform(test_X)

In [ ]:
models = {
    target: LGBMRegressor(verbose=-1).fit(
        train_X_norm, train_y[target],
        eval_set=(valid_X_norm, valid_y[target])
    )
    for target in train_y.columns
}

In [ ]:
def plot_prediction(y, pred):
    plt.scatter(pred, y, alpha=0.2)
    plot_min = min(plt.xlim()[0], plt.ylim()[0])
    plot_max = max(plt.xlim()[1], plt.ylim()[1])
    plt.plot(
        [plot_min, plot_max], [plot_min, plot_max], 
        color="orange", 
        linestyle="dashed",
    )
    plt.title("Prediction vs Real")
    plt.xlabel("Predictions")
    plt.ylabel("Real")
    plt.show()


def regression_report(model, X, y):
    pred = model.predict(X)

    print(f"rmse: {mean_squared_error(y, pred):0.4f}")
    plot_prediction(y, pred)


In [ ]:
for target, model in models.items():
    print(target)
    regression_report(model, train_X_norm, train_y[target]) 

In [ ]:
for target, model in models.items():
    print(target)
    regression_report(model, valid_X_norm, valid_y[target]) 

In [ ]:
for target, model in models.items():
    print(target)
    regression_report(model, test_X_norm, test_y[target]) 

In [ ]:
def run_simulation(models, norm, X: pd.DataFrame, life_window: int):
    curr_X = X
    for _ in range(life_window):
        X_norm = norm.transform(curr_X)
        preds = [
            model.predict(X_norm)
            for _, model in models.items()
        ]
        new_values = np.column_stack(preds) 
        new_features = np.column_stack((new_values, curr_X.iloc[:, :-2].values))
        curr_X = pd.DataFrame(new_features, columns=X.columns)

    return curr_X

In [ ]:
def evaluate_simulation_as_classification(models, norm, X, etc, life_window):
    sim_res = run_simulation(models, norm, X, 30)
    dead_sim = sim_res["Capacity_first_b0"] < EOL_CAPACITY
    dead_rul = etc["rul"] < life_window
    print(classification_report(dead_rul, dead_sim))

In [ ]:
sim_models = {target: models[target] for target in ["time", "cap"]}

In [ ]:
for life in [30, 50, 70]:
    print(f"Life Window: {life}")
    evaluate_simulation_as_classification(sim_models, norm, train_X, train_etc, life)

In [ ]:
for life in [30, 50, 70]:
    print(f"Life Window: {life}")
    evaluate_simulation_as_classification(sim_models, norm, valid_X, valid_etc, life)

In [ ]:
for life in [30, 50, 70]:
    print(f"Life Window: {life}")
    evaluate_simulation_as_classification(sim_models, norm, test_X, test_etc, life)

In [ ]:
def evaluate_simulation_as_regression(models, norm, X, etc):
    max_rul = etc["rul"].max()
    pred_rul = np.full_like(etc["rul"], np.inf, dtype=np.float64)
    pred_rul[X["Capacity_first_b0"] < EOL_CAPACITY] = -np.inf
    curr_X = X
    for i in range(max_rul):
        sim_res = run_simulation(models, norm, curr_X, 1)
        dead = sim_res["Capacity_first_b0"] < EOL_CAPACITY
        pred_rul[dead] = np.minimum(pred_rul[dead], np.full_like(pred_rul[dead], i))
        curr_X = sim_res

    return pred_rul


In [ ]:
train_pred_rul = evaluate_simulation_as_regression(sim_models, norm, train_X, train_etc)
plot_prediction(train_etc["rul"], train_pred_rul)

In [ ]:
valid_pred_rul = evaluate_simulation_as_regression(sim_models, norm, valid_X, valid_etc)
plot_prediction(valid_etc["rul"], valid_pred_rul)

In [ ]:
test_pred_rul = evaluate_simulation_as_regression(sim_models, norm, test_X, test_etc)
plot_prediction(test_etc["rul"], test_pred_rul)